# Python中階-處理刪除文章


## 中正大學資管系 (20181021) 大綱

文章被刪除了，某些資料出現錯誤了？怎麼辦

如果要進一步爬取每一篇文章的內容，那勢必要取得文章的網址，但在爬取過程中會發生一個現象是：文章刪除了導致無法取得網址

![](images/pttdelete.png)

假如以剛剛例子，要用 `entry.find('div.title > a', first=True).attrs['href']` 取的內容，會出現以下錯誤:

```python
AttributeError: 'NoneType' object has no attribute 'attrs'
```

意思是找不到 .attrs 的屬性，很正常，因為被刪除了嘛。那要怎麼處理呢？還記得最一開始講的 `try...except` 例外處理嗎？

In [4]:
import requests
from requests_html import HTML

resp = requests.get('https://www.ptt.cc/bbs/movie/index.html')
html = HTML(html=resp.text)
post_entries = html.find('div.r-ent')
for entry in post_entries:
    title = entry.find('div.title', first=True).text
    print(title)
    link = entry.find('div.title > a', first=True).attrs['href']
    print(link)

[爽無雷] 狂徒-我拜託你們進電影院看!!
/bbs/movie/M.1540011223.A.140.html
[新聞] 巧巧裸戰三級片 傻眼強國妞露毛求上位
/bbs/movie/M.1540012795.A.C11.html
[雷] 電影【請做我的主人】純粹地做愛吧
/bbs/movie/M.1540014076.A.21C.html
(本文已被刪除) [yslimit]


AttributeError: 'NoneType' object has no attribute 'attrs'

In [6]:
for entry in post_entries:
    try:
        title = entry.find('div.title', first=True).text
        print(title)
        link = entry.find('div.title > a', first=True).attrs['href']
        print(link)
    except AttributeError as e: #except Exception as e:
        print('has error')
        break

[爽無雷] 狂徒-我拜託你們進電影院看!!
/bbs/movie/M.1540011223.A.140.html
[新聞] 巧巧裸戰三級片 傻眼強國妞露毛求上位
/bbs/movie/M.1540012795.A.C11.html
[雷] 電影【請做我的主人】純粹地做愛吧
/bbs/movie/M.1540014076.A.21C.html
(本文已被刪除) [yslimit]
has error


### [re](http://www.runoob.com/python/python-reg-expressions.html)

re.search(pattern, string, flags=0)

|   參數  |  描述  |
|---------|-------|
| pattern | 匹配的正則表達式 |
| string  | 要匹配的字符串 |
| flags   | 標誌位，用於控制正則表達式的匹配方式，如：是否區分大小寫，多行匹配等等 |


匹配成功 re.search 方法返回一個匹配的對象，否則返回 None。
我們可以使用 group(num) 或 groups() 匹配對象函數來獲取匹配表達式。

In [13]:
import re

for entry in post_entries:
    try:
        title = entry.find('div.title', first=True).text
        author = entry.find('div.author', first=True).text
        print(title, author)
        link = entry.find('div.title > a', first=True).attrs['href']
        print(link)
    except AttributeError as e: #except Exception as e:
        if '(本文已被刪除)' in title:
            match_author = re.search('\[(\w*)\]', title)
            print(match_author.span())  # 在哪裡開始匹配
            if match_author:
                author = match_author.group(1)
                print(author)
            break
        elif re.search('已被\w*刪除', title):
            match_author = re.search('\<(\w*)\>', title)
            if match_author:
                author = match_author.group(1)

[爽無雷] 狂徒-我拜託你們進電影院看!! mapery
/bbs/movie/M.1540011223.A.140.html
[新聞] 巧巧裸戰三級片 傻眼強國妞露毛求上位 deepdish
/bbs/movie/M.1540012795.A.C11.html
[雷] 電影【請做我的主人】純粹地做愛吧 ariel119
/bbs/movie/M.1540014076.A.21C.html
(本文已被刪除) [yslimit] -
(9, 18)
yslimit


### 稍做解釋正則表達式
(本文已被刪除) [yslimit] 

`re.search('\[(\w*)\]', title)` 

`\[`, `(\w*)` 分開成第二 group 取得裡面 word 最後結尾 `\[`

有篇簡單抓地址的[範例](https://ithelp.ithome.com.tw/articles/10197315)